In [ ]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import json
import bs4
import requests as req
import statistics as st
import operator
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options as CromeOptions
import pickle


utility functions:

In [ ]:
#loads json
def load_json(file_name):
    with open(file_name) as file:
        data = json.load(file)
        return data


#saves to json
def save_to_json(data,file_name):
    with open(file_name,"w",encoding='utf-8') as file:
        json.dump(data,file)


#saves to csv
def save_to_csv(df,file_name):
    df.to_csv(file_name)


#loads from csv
def load_csv(file_name):
    return pd.read_csv(file_name,index_col=0)


#merges 2 dictionaries and returns a combined obw
def merge_dicts(dict1,dict2):
    dict3 = {**dict1,**dict2}
    return dict3


#loads 2 dataframes, combines them, returns and saves the combined df
def load_merge_save_df(file_name1,file_name2,new_file):
    df1 = load_csv(file_name1)
    df2 = load_csv(file_name2)
    df3 = df1.append(df2)
    df3.drop_duplicates(inplace=True)
    save_to_csv(df3,new_file)
    return df3


#loads and merges 2 dataframes
def load_merge_df(df1,file_name2):
    print(type(df1))
    df2 = load_csv(file_name2)
    df3 = df1.append(df2)
    return df3.drop_duplicates()


#gets a list of csv file names and combines them to a new csv file
def load_merge_save_list_csv(file_names,new_file):
    df1 = load_csv(file_names[0])
    for file in file_names:
        df1 = load_merge_df(df1,file)
    save_to_csv(df1,new_file)

creating a dictionary that will store all watched movies

In [ ]:
movie_att_dict = {}

now we will name all of the columns

In [ ]:
Genres_names = ["Action", "Adventure", "Animation", "Comedy", "Crime",
                "Documentary", "Drama", "Family", "Fantasy", "History",
                "Horror", "Music", "Mystery", "Romance", "Science-Fiction",
                "Thriller", "Tv-Movie", "War", "Western"]
user_url =["user_url"]
langs = ["English","Spanish","Portuguese","Japanese","Italian","German","Russian","Arabic",]
countries = ["USA", "Canada", "Australia", "UK","France","Japan","Brazil","Spain","Italy","Germany"]
f_movies = ["favourite 1","favourite 2","favourite 3","favourite 4"]
cast_and_crew = ["favourite director","favourite actor"]
movie_numbers = ["movies watched", "high rated movies"]
years = ["first", "last"]
rating = ["★★★★★","★★★★½","★★★★","★★★½"]
colms =  user_url + f_movies+Genres_names+langs+countries+years+rating+movie_numbers

popular collecting of movies to collect user samples:

In [ ]:
t="https://letterboxd.com/films/genre/"
genres_link = {
    "Action":t+"action/size/small/",
    "Adventure":t+"adventure/size/small/",
    "Animation":t+"animation/size/small/",
    "Comedy":t+"comedy/size/small/",
    "Crime":t+"crime/size/small/",
    "Documentary":t+"documentary/size/small/",
    "Drama":t+"drama/size/small/",
    "Family":t+"family/size/small/",
    "Fantasy":t+"fantasy/size/small/",
    "History":t+"history/size/small/",
    "Horror":t+"horror/size/small/",
    "Music":t+"music/size/small/",
    "Mystery":t+"mystery/size/small/",
    "Romance":t+"romance/size/small/",
    "Science Fiction":t+"science-fiction/size/small/",
    "Thriller":t+"thriller/size/small/",
    "TV Movie":t+"tv-movie/size/small/",
    "War":t+"war/size/small/",
    "Western":t+"western/size/small/"
}
def get_frames_and_urls(driver,genre_link):
    driver.get(genre_link)
    ul = driver.find_elements_by_tag_name("ul")[2]
    a_list = (ul.find_elements_by_tag_name('span'))
    frame_names = []
    for counter,li in enumerate(a_list):
        if(counter%2 == 0):
            frame_names.append(li.get_attribute("textContent"))

    urls = [url.get_attribute("href") for url in ul.find_elements_by_tag_name("a")]
    time.sleep(0.5)
    return frame_names,urls

In [ ]:
browser= webdriver.Chrome()
list_ = list(genres_link.values())
all_genres_links = []
for link in list_:
    print(link)
    frame_list,link_list = get_frames_and_urls(browser, link)
    all_genres_links.extend(link_list)
    time.sleep(0.5)

from those movies we will now collect the user sample using beautiful soup

In [ ]:
members = set()
for counter,movie in enumerate(all_genres_links):
    request = req.get(f'{movie}/ratings/')
    html_page = request.text
    bs = bs4.BeautifulSoup(html_page,'html.parser')
    a_list = bs.find("ul",class_ = "avatar-list -a40 -cols16").find_all('a',href=True)
    for i,a in enumerate(a_list):
        members.add(a_list[i]["href"])

    print(f'{counter+1}:843')
with open('members.csv','w') as f:
    for member in members:
        f.write('%s,' % member)

we collected 500 users from each which translated to almost 200k users so obviously we had to use a small sample of the users

crawling functions:
movie_crawl_attr - takes relevant data from each movie
user_scan - scans user reviewed movies and returns values for the relevant categories
user_info - gets each users URL and returns a df row containing the users relevant data

In [ ]:
def movie_crawl_attr(driver1, frame_title,url):
    try:
        return movie_att_dict[frame_title]
    except KeyError:
        driver1.get(url)
        time.sleep(2)
        driver1.execute_script("window.scrollBy(0,750)")
        xpath_list = ['//*[@id="tab-cast"]/div/p/a[1]','//*[@id="tab-details"]/h3','//*[@id="tab-genres"]/div']
        new_dict = {}
        categories = []
        runs = len(driver1.find_elements(By.XPATH,'//*[@id="tabbed-content"]/header/ul/li'))
        for i in range(1,5):
            if i > runs:
                break
            elem = WebDriverWait(driver1,10).until(EC.element_to_be_clickable((By.XPATH,f'//*[@id="tabbed-content"]/header/ul/li[{i}]/a')))
            elem.click()
            category = elem.get_attribute("textContent")
            categories.append(category)
            if(category == "Cast"):#cast
                try:
                    new_dict["leading_actor"]=WebDriverWait(driver1,10).until(EC.visibility_of_element_located((By.XPATH,xpath_list[0]))).text
                except:
                    new_dict["leading_actor"]=''
            elif(category == "Crew"): #crew
                pass
            elif(category == "Details"): #details
                tabs = driver1.find_elements(By.XPATH,xpath_list[1])
                for j,tab in enumerate(tabs):
                    if tab.get_attribute("textContent")=="Country" or tab.get_attribute("textContent") == "Countries":
                        list_l=WebDriverWait(driver1,10).until(EC.presence_of_element_located((By.XPATH,f'//*[@id="tab-details"]/div[{j+1}]/p')))
                        new_dict["country"]= [l.text for l in list_l.find_elements_by_tag_name("a")]
                    elif tab.get_attribute("textContent")=="Language" or tab.get_attribute("textContent")=="Languages":
                        list_l=WebDriverWait(driver1,10).until(EC.presence_of_element_located((By.XPATH,f'//*[@id="tab-details"]/div[{j+1}]/p')))
                        new_dict["language"]= [l.text for l in list_l.find_elements_by_tag_name("a")]
                here = [h.get_attribute("textContent") for h in tabs]
                if "Countries" not in here and "Country" not in here:
                    new_dict["country"]=[]
                if "Languages" not in here and "Language" not in here:
                    new_dict["language"]=[]
            elif(category == "Genres" or category == "Genre"):#genres
                try:
                    list_l=WebDriverWait(driver1,10).until(EC.presence_of_element_located( (By.XPATH,xpath_list[2])))
                    new_dict["genres"]=[l.text for l in list_l.find_elements_by_tag_name("a")]
                except:
                    new_dict["genres"]=[]
                break
        if len(categories) < 4:
            if len(categories) == 3 and "Crew" not in categories:
                pass
            else:
                if "Cast" not in categories:
                    new_dict["leading_actor"]=''
                if "Details" not in categories:
                    new_dict["country"]=[]
                    new_dict["language"]=[]
                if "Genres" not in categories and "Genre" not in categories:
                    new_dict["genres"] = []
        else:
            pass
        p = driver1.find_element_by_xpath('//*[@id="featured-film-header"]/p')
        list_a = p.find_elements_by_tag_name('a')
        try:
            new_dict['director'] = list_a[1].text
        except:
            new_dict['director'] = ""
        new_dict['year'] = int(list_a[0].text)
        # adding new movie attributes dicts to the main dict
        movie_att_dict[frame_title] = new_dict
        return  new_dict

In [ ]:
def user_scan(driver,url):
    genre_dict = {gnr: 0 for gnr in Genres_names }
    lang_dict = {lng: 0 for lng in langs}
    countries_dict = {cntr:0 for cntr in countries}
    leading_actors = {}
    directors = {}
    num_of_high_rated = 0
    year_list = []
    name_link_dict,rating_dict = user_movies(driver,url)
    for name in name_link_dict.keys():
        movie_release =int(name[len(name)-5:len(name)-1])
        if movie_release >= 1948:
            this_movie = movie_crawl_attr(driver, name,name_link_dict[name])
            if len(movie_att_dict[name]["language"]) > 0 and movie_att_dict[name]["language"] == '':
                while len(movie_att_dict[name]["language"]) > 0 and movie_att_dict[name]["language"] == '':
                    del(movie_att_dict[name])
            year_list.append(movie_release)
            for l in this_movie["language"]:
                try:
                    lang_dict[l]+=1
                except KeyError:
                    pass
            for c in this_movie["country"]:
                try:
                    countries_dict[c]+=1
                except KeyError:
                    pass
            for g in this_movie["genres"]:
                genre_dict[g]+=1
            try:
                leading_actors[this_movie["leading_actor"]]+=1
            except KeyError:
                leading_actors[this_movie["leading_actor"]]=1
            try:
                directors[this_movie["director"]]+=1
            except KeyError:
                directors[this_movie["director"]]=1
            num_of_high_rated+=1
            year_list.append(this_movie["year"])
        else:
            pass
    year_max = max(year_list)
    year_min = min(year_list)
    year_mean = st.mean(year_list)
    year_mid = st.median(year_list)
    fav_actor = max(leading_actors.items(),key = operator.itemgetter(1))
    fav_director = max(directors.items(),key = operator.itemgetter(1))
    return genre_dict, lang_dict, countries_dict, num_of_high_rated, year_max, year_min, year_mean, year_mid, fav_actor, fav_director, rating_dict

In [ ]:
def user_info(driver, user_url):
    url = f"https://letterboxd.com{user_url}"
    driver.get(url)
    user_v = pd.DataFrame(index=[driver.find_element_by_class_name("title-1").get_attribute("textContent")],columns=colms)
    user_v["user_url"] = user_url
    user_v["movies watched"]= int(driver.find_element_by_class_name("value").get_attribute("textContent").replace(',',''))
    try:
        favs = driver.find_element_by_id("favourites")
        for number , mover in enumerate(favs.find_elements_by_class_name("frame-title")):
            user_v[f"favourite {number+1}"] = mover.get_attribute("textContent")
    except:
        for number in range(4):
            user_v[f"favourite {number+1}"] = ""
    flag = 1
    while flag == 1:
        try:
            genre_dict, lang_dict, countries_dict, num_of_high_rated, year_max, year_min, year_mean, year_mid, fav_actor, fav_director, rating_dict =user_scan(driver,f'{url}films/ratings/by/entry-rating/')
            flag = 0
        except ValueError:
            pass

    for key in genre_dict.keys():
        user_v[key]=genre_dict[key]
    for key in lang_dict:
        user_v[key]= lang_dict[key]
    for key in countries_dict.keys():
        user_v[key] = countries_dict[key]
    for key in rating_dict:
        user_v[key] = rating_dict[key]
    user_v["first"] = year_min
    user_v["last"] = year_max
    user_v["mean"] = year_mean
    user_v["mid"] = year_mid
    user_v["high rated movies"]= num_of_high_rated
    return user_v

In [ ]:
def user_movies(driver,url):
    flag = 1
    driver.get(url)
    links_names = {}
    rating_dict = {}
    flag2 = 1
    while flag == 1:
        next_list = driver.find_elements_by_class_name("next")
        if next_list == []:
            flag = 0
        else:
            next_link = next_list[0].get_attribute("href")
            if not next_link:
                flag = 0
        try:
            frames_list =  driver.find_element(By.XPATH,'//*[@id="content"]/div/div/section/ul')
            frame_links = [a.get_attribute("href") for a in frames_list.find_elements_by_tag_name('a')]
            frame_names = [fn.get_attribute("textContent") for fn in driver.find_elements_by_class_name("frame-title")]
            frame_rates = [rate.find_element(By.TAG_NAME,"span").get_attribute("textContent").strip() for rate in driver.find_elements(By.CLASS_NAME,"poster-viewingdata")]
            flag2 = 1
        except:
            flag2 = 0

        if flag2 == 1:
            for name,link,rate in zip(frame_names,frame_links,frame_rates):
                if len(rate)>3:#getting the nuber of stras for this movie
                    links_names[name] = link
                    try:
                        rating_dict[rate]+=1
                    except KeyError:
                        rating_dict[rate] = 1
                    #print(name.get_attribute("textContent"))
                else:
                    flag = 0
                    break
        else:
            pass
        if flag == 1 and flag2 == 0:
            driver.find_element_by_tag_name('body').send_keys(Keys.COMMAND + 'r')
            flag2 = 1
        elif flag == 1 and flag2 == 1:
            driver.get(next_link)
        else:
            return links_names, rating_dict

open_driver - opens a new chrome driver and automatically downloads wanted cookies and adblock

In [ ]:
def open_driver():
    #was done with the help of youtube channel Computing HUB
    #https://www.youtube.com/watch?v=Fx1hbZMVS7k
    ops = CromeOptions()
    ops.add_extension("adblock.crx")
    browser = webdriver.Chrome(options = ops)
    time.sleep(5)
    ##close new tab
    browser.switch_to.window(browser.window_handles[1])
    browser.close()
    browser.switch_to.window(browser.window_handles[0])
    browser.get("https://letterboxd.com/")
    cookies = pickle.load(open("cookies2.pkl", "rb"))
    browser.maximize_window()
    for cookie in cookies:
        browser.add_cookie(cookie)
    return browser

creates new df to add

In [ ]:
df = pd.DataFrame(index=[],columns=colms)

this is where the magic happens, this is the crawling block, here we go from one user to another, save their data in a our dataframe and save our progress once every 50 users, better safe then sorry!

In [ ]:
browser = open_driver()
for i, mem in enumerate(members):
    if i % 50 == 0:
        save_to_csv(df,f"df_{i}.csv")
        save_to_json(movie_att_dict,f"movies_{i}.json")
    if req.get(f"https://letterboxd.com{mem}").status_code != 404:
        t_df = user_info(browser, mem)
        df = df.append(t_df)
    else:
        pass
    print(f"{i} done! {mem}")

that's it! now we can go on to the preprocessing and machine learning